In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## Часть 1

#### Get books html elements

In [7]:
base_url = "https://www.litres.ru/genre/programmirovanie-5272/?page={}"

books = []
books_count = 960
page = 1

while len(books) < books_count:
    url = base_url.format(page)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    books_on_page = soup.find_all('div', class_=re.compile(r'ArtDefault_container'))

    remaining_books = books_count - len(books)
    books.extend(books_on_page[:remaining_books])

    page += 1
    time.sleep(1)

In [8]:
def clean_html(raw_html: str) -> str:
    soup = BeautifulSoup(raw_html, "html.parser")
    return soup.get_text()


def get_reviews(book_id: str):
    url = f"https://api.litres.ru/foundation/api/arts/{book_id}/reviews"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()

        if data["status"] == 200 and "data" in data["payload"]:
            reviews = data["payload"]["data"]
            review_texts = [clean_html(review["text"]) for review in reviews]
            return review_texts
        else:
            print("No reviews found or incorrect response structure.")
            return []
    else:
        print(f"Failed to retrieve reviews. Status code: {response.status_code}")
        return []

In [9]:
def parse_book_id(url: str) -> str:
    book_id = url.rstrip('/').split('-')[-1]
    return book_id


def get_book_html(book_url: str):
    response = requests.get(book_url)
    soup = BeautifulSoup(response.text, "html.parser")

    review_div = soup.find('div', {'data-testid': 'book-factoids__reviews'}).text.strip()
    review_count = int(re.search(r'\d+', review_div).group()) if review_div else 0

    details_div = soup.find('div', {'data-testid': 'book-volume__wrapper'}).text.strip()
    pages_count_match = re.search(r'Объем\s(\d+)\sстраниц', details_div)
    pages_count = int(pages_count_match.group(1)) if pages_count_match else 0

    year_match = re.search(r'(\d{4})\sгод', details_div)
    year = int(year_match.group(1)) if year_match else 0

    age_match = re.search(r'(\d+)\+', details_div)
    age = int(age_match.group(1)) if age_match else 0

    id = parse_book_id(book_url)
    text_reviews = get_reviews(id)
    time.sleep(1)

    return review_count, pages_count, text_reviews, age, year

#### Fetch books data

In [11]:
data = []

for book in books:
    name = book.find('a', {'data-testid': 'art__title'}).text.strip()

    author_a = book.find('a', {'data-testid': 'art__authorName'})
    author = author_a.text.strip() if author_a else ''

    link = book.find('div', {'data-testid': 'art__cover'})
    full_link = f"https://www.litres.ru{link.find('a')['href']}"

    rating_div = book.find('div', {'data-testid': 'art__ratingAvg'}).text.strip()
    rating = float(rating_div.replace(',', '.'))

    rating_count_div = book.find('div', {'data-testid': 'art__ratingCount'})
    rating_count = int(rating_count_div.text) if rating_count_div else 0

    price_strong = book.find('strong', {'data-testid': 'art_price--value'})
    price = int(re.sub(r'\D', '', price_strong.text.strip())) if price_strong else 0

    review_count, pages_count, text_reviews, age, year = get_book_html(full_link)

    data.append({
        'name': name,
        'author': author,
        'link': full_link,
        'rating': rating,
        'rating_count': rating_count,
        'review_count': review_count,
        'pages_count': pages_count,
        'text_reviews': text_reviews,
        'price': price,
        'age': age,
        'year': year,
    })

AttributeError: 'NoneType' object has no attribute 'text'

In [ ]:
df = pd.DataFrame(data)
df.head()

## Часть 2

1. Выведите первые 5 строк датасета. (0.25). Сколько в нём строк и столбцов (0.25)?

In [ ]:
rows, columns = df.shape
print(f'Строк: {rows}, столбцов: {columns}')

2. Есть ли в датасете пропуски? (0.5)

In [ ]:
missing_values = df.isnull().any().any()

if missing_values:
    print('В датасете есть пропущенные значения.')
else:
    print('В датасете нет пропущенных значений.')

3. Проверьте типы данных. Если это необходимо, приведите к типам int и float те столбцы, с которыми понадобится работать как с числами. (1).

In [ ]:
df.dtypes

4. Выведите описательные статистики переменных. Ответьте на следующие вопросы:

* Какая медианная цена книги в вашем датасете? (1)  
* Какое возрастное ограничение встречается чаще всего? (1)  
* Какое среднее число отзывов в книге? (1)  
* Сколько книг имеют оценку ниже 4.25? (1)  
* В каком году было написано больше всего книг из датасета? (1)  

In [ ]:
price_median = df['price'].median()
age_mode = df['age'].mode()[0]
mean_reviews_count = df['review_count'].mean()
low_rating_count = df[df['rating'] < 4.25].shape[0]

year_counts = df['year'].value_counts()
most_common_year = year_counts.idxmax()

print(f'Медианная цена книги: {price_median}')
print(f'Самое частое возрастное ограничение: {age_mode}')
print(f'Среднее число отзывов в книге: {mean_reviews_count}')
print(f'Число книг имеют оценку ниже 4.25: {low_rating_count}')
print(f'Больше всего книг в году: {most_common_year}')

5. Если вы работаете с готовым датасетом, то попробуйте "достать" из столбца pages количество страниц. Если у вас не получилось, то далее при определении числа страниц пользуйтесь стольцов pages_count. Если вы парсили датасет сами, то вы получаете балл за этот пункт автоматически (1.5)

In [ ]:
df.pages_count

6. Создайте новое поле is_popular. Значение равно 1, если рейтинг книги не менее 4.6 и при этом у нее не менее 5 отзывов, и 0 в остальных случаях. (1)

In [ ]:
df['is_popular'] = np.where((df['rating'] >= 4.6) & (df['review_count'] >= 5), 1, 0)
df.head()

7. Как отличается среднее число страниц среди популярных и непопулярных книг? (2)

In [ ]:
difference = abs(df[df['is_popular'] == 1]['pages_count'].mean() - df[df['is_popular'] == 0]['pages_count'].mean())

print(f"Разница в среднем числе страниц между популярными и непопулярными книгами: {difference:.2f}")

8. Выведите топ-10 книг по числу отзывов. (2).

In [ ]:
df.sort_values(by='review_count', ascending=False).head(10)

9. Найдите среднюю длину отзыва (в символах). (2)

In [ ]:
def average_review_length(reviews):
    if len(reviews) == 0:
        return 0
    total_length = sum(len(review) for review in reviews)
    return total_length / len(reviews)


# Вычисляем среднюю длину отзыва для каждой книги
df['average_review_length'] = df['text_reviews'].apply(average_review_length)

# Вычисляем среднюю длину отзыва по всем книгам
overall_average_length = df['average_review_length'].mean()

print(f"Средняя длина отзыва по всем книгам: {overall_average_length:.2f} символов")

10. Постройте таблицу корреляций числовых переменных. (1) Прокомментируйте результаты. (1)

In [ ]:
numeric_df = df.select_dtypes(include='number')
corr_matrix = numeric_df.corr()

In [ ]:
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")

Согласно heatmap наиболее коррелируют между собой следующие параметры:
- Число оценок и число отзывов (положительная корреляция)
- Число оценок и популярность (положительная корреляция)
- Число отзывов и популярность (положительная корреляция, но не такая сильная)

Из забавного есть небольшая отрицательная корреляция между кол-вом страниц и числом оценок =) 

11. Постройте диаграмму рассеяния (scatterplot) количества страниц и количества отзывов. Не забудьте подписать график и оси. (1) Прокомментируйте полученные результаты. (1)

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(x='pages_count', y='review_count', data=df)

plt.title('Диаграмма рассеяния: Количество страниц и Количество отзывов')
plt.xlabel('Количество страниц')
plt.ylabel('Количество отзывов')

plt.show()

- Большинство книг сосредоточены в диапазоне до 600 страниц и до 10 отзывов.
- Наблюдается некоторая положительная корреляция между количеством страниц и количеством отзывов, но она не является строгой. Это значит, что книги с большим количеством страниц могут получать больше отзывов, но это не правило, и другие факторы могут играть важную роль.

12. Постройте линейный график: по оси Х год, по оси Y количество книг. (1) Прокомментируйте. (1)

In [ ]:
df_filtered = df[df['year'] > 0]
books_per_year = df_filtered.groupby('year').size()

plt.figure(figsize=(10, 6))
books_per_year.plot(kind='line', marker='o')

plt.title('Количество книг по годам')
plt.xlabel('Год')
plt.ylabel('Количество книг')
plt.grid(True)

plt.show()

- До 2015 года стабильно выпускалось по одной книге
- Пик приходится на 2023 год, в 2024 на 2 книги выпущено меньше

12. Постройте еще любые два графика по вашему усмотрению. (2) Прокомментируйте полученные результаты. (1.5)

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(df['rating'], bins=5, edgecolor='black')
plt.title('Распределение рейтингов книг')
plt.xlabel('Рейтинг')
plt.ylabel('Количество книг')
plt.grid(True)
plt.show()

Большинство книг имеент оценку 4 и больше

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(x=df['review_count'])
plt.title('Распределение количества отзывов на книги')
plt.xlabel('Количество отзывов')
plt.show()

Большинство книг имеет количество отзывов, сосредоточенное в нижней части шкалы, что говорит о том, что большинство книг в выборке имеют относительно мало отзывов, в то время как несколько книг получили значительно больше внимания.

13. Постройте таблицу с авторами книг с именем автора, количество книг в датасете, средней оценкой книг, средним количеством отзывов. (2).

In [ ]:
authors_table = df.groupby('author').agg(
    books_count=('name', 'count'),
    average_rating=('rating', 'mean'),
    average_review_count=('review_count', 'mean')
).reset_index()
print(authors_table)

14. Что еще интересного можно увидеть в этом датасете? Просмотрите на данные и ответьте на какие-нибудь вопросы, на которые не ответили в предыдущим пункте. Мы никак не ограничиваем вашу фантазию! (3).

In [ ]:
authors_page_count = df.groupby('author').agg(
    average_pages=('pages_count', 'mean')
).reset_index()
authors_page_count = authors_page_count.sort_values(by='average_pages', ascending=False)

print('Какие авторы пишут самые объемные книги?')
print(authors_page_count)

In [ ]:
df['reviews_to_rating_ratio'] = df['review_count'] / df['rating']
top_discussed_books = df.sort_values(by='reviews_to_rating_ratio', ascending=False).head(10)

print('Какие книги являются самыми обсуждаемыми (имеют наибольшее количество отзывов) по отношению к их рейтингу?')
print(top_discussed_books[['name', 'author', 'rating', 'review_count', 'reviews_to_rating_ratio']])